# Example of using LightPipelines on Spark OCR
## Install spark-ocr python packge
Need specify path to `spark-ocr-assembly-[version].jar` or `secret`

In [1]:
license = ""
secret = ""
nlp_secret = ""
aws_access_key = ""
aws_secret_key = ""

version = secret.split("-")[0]
spark_ocr_jar_path = "../../target/scala-2.12"

In [2]:
%%bash
if python -c 'import google.colab' &> /dev/null; then
    echo "Run on Google Colab!"
    echo "Install Open JDK"
    apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
    java -version
fi

In [3]:
import sys
import os

if 'google.colab' in sys.modules:
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

## Initialization of spark session

In [5]:
from pyspark.sql import SparkSession
from sparkocr import start

if license:
    os.environ['JSL_OCR_LICENSE'] = license
    

spark = start(secret=secret,
              nlp_secret=nlp_secret,
              jar_path=spark_ocr_jar_path,
              nlp_internal="4.2.3")

Spark version: 3.1.2
Spark NLP version: 4.2.2
Spark NLP for Healthcare version: 4.2.1
Spark OCR version: 4.2.1rc3



In [6]:
from pyspark.ml import PipelineModel
from pyspark.sql.functions import *
from sparkocr.transformers import *

In [7]:
import pkg_resources
pdfs_path = pkg_resources.resource_filename('sparkocr', 'resources/ocr/pdfs/four_pages_clinical_note')


## Define OCR pipeline

In [9]:
# Transform binary to image
pdf_to_image = PdfToImage()
pdf_to_image.setOutputCol("image")

# Run OCR for each region
ocr = ImageToText()
ocr.setInputCol("image")
ocr.setOutputCol("text")
ocr.setConfidenceThreshold(60)

# OCR pipeline
pipeline = PipelineModel(stages=[
    pdf_to_image,
    ocr
])

## Create LightPipeline

In [10]:
from sparkocr.base import LightPipeline

In [11]:
lp = LightPipeline(pipeline)

In [12]:
%%time
lp.fromLocalPath(pdfs_path)

CPU times: user 4.58 ms, sys: 6.95 ms, total: 11.5 ms
Wall time: 6.24 s


[{'text': [Annotation(ImageToText, 0, 1752, Result Information
   
    
   
    
   
   Status Provider Status
   Final result iy Reviewed
   Entry Date
   
    
   
   Component Results
   
   Component Lab
   Surgical Pathology
   
   Patient Nam
   MR#: JCD-O-3
   Specime
   Ca rCtnome unshelia| NOS
   
   Final Diagnosis ZBIAO/B3
   
   A. Right ureter, biopsy: x
   -Segment of benign ureter, negative for tumor Sitdez Lb lad cNer ,wall, m5
   -Frozen section diagnosis confirmed
   
   Cw7.4
   
   B. Left ureter, biopsy: B27 IU
   -Segment of benign ureter, negative for tumor #0
   -Frozen section diagnosis confirmed
   
   C. Left external and internal lymph nodes, resection:
   ~Benign fibrovascular tissue, negative for tumor
   -No lymph nodes identified
   
   D. Left internal and external iliac lymph node, resection:
   -Single lymph node with metastatic urothelial carcinoma, 1.8 cm largest diameter
   (1/1)
   
   E. Left internal and external iliac lymph nodes, resection:
  

## Integration with Spark-NLP pipelines

In [13]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols("document")\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols("sentence")\
  .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
  .setInputCols(["sentence","token"])\
  .setOutputCol("embeddings")

ner = MedicalNerModel.pretrained("ner_jsl", "en","clinical/models")\
  .setInputCols(["sentence","token","embeddings"])\
  .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols("sentence", "token", "ner")\
  .setOutputCol("entities")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl download started this may take some time.
[OK!]


## Mixed Pipeline

In [14]:
# OCR pipeline
from pyspark.ml import Pipeline, PipelineModel
pipeline = Pipeline(stages=[
    pdf_to_image,
    ocr,
    documentAssembler,
    sentenceDetector,
    tokenizer,
    embeddings,
    ner,
    converter
])

empty_data = spark.createDataFrame([[b'', ""]]).toDF("content", "path")
pipeline = pipeline.fit(empty_data)
lp = LightPipeline(pipeline)

In [15]:
result = lp.fromLocalPath(pdfs_path)

In [16]:
import pandas as pd
import itertools as it

single_page = result[0]
p_df = pd.DataFrame(list(zip(it.cycle(single_page["text"]), it.cycle(single_page["sentence"]), 
                           single_page["entities"])),
                 columns=["Text", "Sentence", "Entities"])

p_df

,Text,Sentence,Entities
0,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 0, 1267, Result Informati...","Annotation(chunk, 230, 234, Right, Map(entity ..."
1,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 1269, 1624, Right obturat...","Annotation(chunk, 236, 241, ureter, Map(entity..."
2,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 1626, 1715, involving bla...","Annotation(chunk, 244, 249, biopsy, Map(entity..."
3,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 1717, 1718, 8., Map(sente...","Annotation(chunk, 266, 278, benign ureter, Map..."
4,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 0, 1267, Result Informati...","Annotation(chunk, 294, 298, tumor, Map(entity ..."
5,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 1269, 1624, Right obturat...","Annotation(chunk, 320, 323, wall, Map(entity -..."
6,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 1626, 1715, involving bla...","Annotation(chunk, 330, 353, Frozen section dia..."
7,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 1717, 1718, 8., Map(sente...","Annotation(chunk, 376, 379, Left, Map(entity -..."
8,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 0, 1267, Result Informati...","Annotation(chunk, 381, 386, ureter, Map(entity..."
9,"Annotation(ImageToText, 0, 1752, Result Inform...","Annotation(document, 1269, 1624, Right obturat...","Annotation(chunk, 389, 394, biopsy, Map(entity..."
